# Imports

In [4]:
# Default ds/de imports
import pandas as pd

# MetaTrader5
import MetaTrader5 as mt5

# Web scraping
import requests
from bs4 import BeautifulSoup

# Functions

In [11]:
def get_small_companies(market_cap_limit=1_000_000_000) -> pd.DataFrame:
    """
    Fetches stock data from Fundamentus website, cleans it, and filters for small cap companies.
    
    Args:
        market_cap_limit: Maximum market cap value to consider a company as small cap
        
    Returns:
        DataFrame containing filtered small cap companies
    """
    # Scrape Fundamentus main table
    url = "https://www.fundamentus.com.br/resultado.php"
    
    # Headers needed to avoid 403 Forbidden
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
    }
    
    response = requests.get(url, headers=headers)
    response.encoding = "latin-1"
    
    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find("table")

    # Parse the HTML table into DataFrame
    df = pd.read_html(str(table), decimal=",", thousands=".")[0]
    
    # Clean column names (remove spaces, lowercase)
    df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]
    
    # Filter by desired rules (small caps)
    # Remove NaNs
    df = df.dropna(subset=["p/ativo"])

    # Filter by market cap limit
    small_caps_df = df[df["p/ativo"] <= market_cap_limit]
    
    return small_caps_df

def get_market_book(symbol):
    """
    Get market book data for a specified symbol.
    
    Args:
        symbol (str): The trading symbol to get market book data for
        
    Returns:
        list or None: Market book data if successful, None otherwise
    """
    if not mt5.symbol_select(symbol, True):
        print(f"symbol_select() failed for {symbol}, error code = {mt5.last_error()}")
        return None
    
    book = mt5.market_book_get(symbol)
    return book

## Getting data from mql5

In [12]:
# Initialize MetaTrader5
if not mt5.initialize():
    print("initialize() failed, error code =", mt5.last_error())
    quit()

In [13]:
# Example usage
symbol = "TASA3"
book = get_market_book(symbol)


In [14]:
book

## Portfolio definition

In [ ]:
# Get small caps
small_caps = get_small_companies()